
# CIFAR-10 Unlearning Report — Delta Aggregation by `unlearn_epochs`

This notebook reads a results CSV (e.g. `CIFAR10_BTI_DBF_U_results.csv`) and, for each distinct value of `unlearn_epochs`, reports:
- **Average change in benign accuracy:** `benign_accuracy - orig_benign_accuracy`
- **Average change in attack success rate:** `attack_success_rate - orig_attack_success_rate`
- **Average time taken to clean a model**

It also saves the aggregated table to `unlearn_epoch_deltas.csv` alongside the notebook.

Created with AI assitance from ChatGPT5


In [1]:
# --- Parameters ---
# Set the column to group / separate the analysis by.
# Examples: 'unet_tau', 'unet_lr', 'unlearn_epochs', 'variant_name', 'split_mode'
GROUP_BY = 'experiment_name'


In [2]:

import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Display options for pandas
pd.set_option('display.width', 120)
pd.set_option('display.max_columns', None)


In [3]:

# === Configuration ===
# Set your CSV path here. Example: 'CIFAR10_BTI_DBF_U_results.csv'
CSV_PATH = './MNIST_BTI_DBF_U_results.csv'
OUTPUT_SUMMARY = 'unlearn_epoch_deltas.csv'

# If your file isn't comma-separated, we will let pandas infer the delimiter.
# If inference fails, set sep explicitly, e.g., sep='\t' for TSV.

## Parameterized grouped report (with average elapsed seconds)

In [4]:
# --- Grouped report (parameterized by GROUP_BY) ---
import os, glob
import pandas as pd

# Try to reuse a path from the notebook if it's defined (RESULTS_CSV); otherwise auto-detect.
try:
    results_path = CSV_PATH  # noqa: F821
except NameError:
    candidates = sorted(glob.glob("*_BTI_DBF_U_results.csv"))
    if len(candidates) == 0:
        raise FileNotFoundError("No *_BTI_DBF_U_results.csv found in working directory and RESULTS_CSV not defined.")
    results_path = candidates[0]

df = pd.read_csv(results_path)

# Compute changes (if columns exist)
# Compute changes (if columns exist)
if {'orig_benign_accuracy','benign_accuracy'}.issubset(df.columns):
    df['delta_ba'] = df['benign_accuracy'] - df['orig_benign_accuracy']
if {'orig_attack_success_rate','attack_success_rate'}.issubset(df.columns):
    df['delta_asr'] = df['attack_success_rate'] - df['orig_attack_success_rate']

# --- NEW: mask deltas ---
if {'orig_benign_accuracy','mask_benign_accuracy'}.issubset(df.columns):
    df['mask_delta_ba'] = df['mask_benign_accuracy'] - df['orig_benign_accuracy']
if {'orig_attack_success_rate','mask_attack_success_rate'}.issubset(df.columns):
    df['mask_delta_asr'] = df['mask_attack_success_rate'] - df['orig_attack_success_rate']

# Build aggregation dictionary dynamically based on available columns
agg_dict = {}
for col in [
    'delta_ba', 'delta_asr', 'elapsed_seconds',
    'benign_accuracy','attack_success_rate','overall_accuracy',
    'mask_delta_ba','mask_delta_asr', 'mask_elapsed_seconds',
    'mask_benign_accuracy','mask_attack_success_rate','mask_overall_accuracy'
]:
    if col in df.columns:
        agg_dict[col] = 'mean'

# Group
summary = df.groupby(GROUP_BY, dropna=False).agg(agg_dict).reset_index()

# Rename for clarity
summary = summary.rename(columns={
    'delta_ba': 'avg_delta_ba',
    'delta_asr': 'avg_delta_asr',
    'elapsed_seconds': 'avg_elapsed_seconds',
    'benign_accuracy': 'avg_benign_accuracy',
    'attack_success_rate': 'avg_attack_success_rate',
    'overall_accuracy': 'avg_overall_accuracy',
    'mask_delta_ba': 'avg_mask_delta_ba',
    'mask_delta_asr': 'avg_mask_delta_asr',
    'mask_elapsed_seconds': 'avg_mask_elapsed_seconds'
    'mask_benign_accuracy': 'avg_mask_benign_accuracy',
    'mask_attack_success_rate': 'avg_mask_attack_success_rate',
    'mask_overall_accuracy': 'avg_mask_overall_accuracy',
})

# Sort summary by GROUP_BY
summary = summary.sort_values(by=[GROUP_BY]).reset_index(drop=True)

try:
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user(name=f"Grouped report by {GROUP_BY}", dataframe=summary)
except Exception:
    display(summary)

# Save summary
out_path = os.path.splitext(results_path)[0] + f"_grouped_by_{GROUP_BY}.csv"
summary.to_csv(out_path, index=False)
print(f"Saved grouped summary to {out_path}")

,experiment_name,avg_delta_ba,avg_delta_asr,avg_elapsed_seconds,avg_benign_accuracy,avg_attack_success_rate,avg_overall_accuracy
0,BTIDBFU-Fast,-0.593059,-89.902333,77.444267,98.776,8.174,0.96751
1,BTIDBFU-Fast-new,-0.610000,-91.136667,81.728040,98.800,7.060,0.97265
2,BTIDBFU-Standard,-0.443059,-90.912333,237.362535,98.926,7.164,0.97562


Saved grouped summary to ./MNIST_BTI_DBF_U_results_grouped_by_experiment_name.csv
